In [ ]:
cd /data/store01/data_dirs/students/ucfafyi/Sentinel

In [ ]:
#Amazon-ATTO-Tower,-2.143889,-59.000
import sys                        
sys.path.insert(0, 'python')
from optimizing import *
from Dload_Finder import *
import gdal
import pylab as plt
%matplotlib inline

In [3]:
lat, lon, year = -2.143889,-59.0, 2016
sfiles, mfiles = get_closet(lat, lon,year,0)

In [5]:
spath = sfiles[0]; mfile = mfiles[0]
print spath, mfile

/home/ucfafyi/DATA/Sentinel/s_data/21/M/TT/2016/1/9/0/ [ '/home/ucfafyi/DATA/Sentinel/m_data/MCD43A1.A2016009.h12v09.005.2016026083209.hdf'
 '/home/ucfafyi/DATA/Sentinel/m_data/MCD43A2.A2016009.h12v09.005.2016026083210.hdf']


In [33]:
import sys
sys.path.insert(0,'python')
import numpy.ma as ma
from collections import Counter
import cPickle as pkl
from scipy import optimize
from functools import partial
import scipy.ndimage as ndimage
import numpy as np
import scipy
from scipy import signal
import scipy.stats
from downloader_and_file_finder import *
from cloud import *
from fastRWpkl import *
import numpy 
from numpy import clip, where
from scipy.ndimage.morphology import *
import xml.etree.cElementTree as ET
import multiprocessing
from get_r import *
from sklearn import linear_model

keys = 'B02', 'B03','B04','B08','B8A','B11','B12'
bands = [2,3,4,8,13,11,12]

def gaussian(xwin, ywin, xstd, ystd, angle, norm = True):
    win = max(xwin, ywin)
    winx = win*2**0.5
    winy = win*2**0.5
    
    xstd = xstd*2**0.5
    ystd = ystd*2**0.5
        
    xgaus = signal.gaussian(winx, xstd)
    ygaus = signal.gaussian(winy, ystd)
    gaus  = np.outer(xgaus, ygaus)
    r_gaus = scipy.ndimage.interpolation.rotate(gaus, angle, reshape=True)
    center = np.array(r_gaus.shape)/2
    cgaus = r_gaus[center[0]-xwin/2: center[0]+xwin/2, center[1]-ywin/2:center[1]+ywin/2]
    if norm:
        return cgaus/cgaus.sum()
    else:
        return cgaus

def cost(p, sent, sinds, mod, minds, band):    
    xstd,ystd,angle, xs, ys = p
    xwin,ywin = 150, 150
    
    to_regression =[]          
    cx = sinds[0]
    cy = sinds[1]
    mx = minds[0]
    my = minds[1]
    
    gaus = gaussian(xwin,ywin,xstd,ystd,angle,False)                              
    ker = gaus/(gaus.sum())

    s = signal.fftconvolve(sent, ker, mode='same')
    
    vld_x = ((cx+xs)>xwin/2)&((cx+xs)<10000-xwin/2)
    vld_y = ((cy+ys)>ywin/2)&((cy+ys)<10000-ywin/2)
    vld = vld_x&vld_y
    
    
    indx,indy = np.round((cx+xs)[vld]).astype(int), np.round((cy+ys)[vld]).astype(int)
    vals = s[indx,indy]
    brdf = mod[mx[vld], my[vld]]
    mask = (brdf>0)&(brdf<1)&(vals>0)&(vals<1)
    
    x,y = ransaclin(vals[mask], brdf[mask])
    
    m = y.ravel()
    s = x.ravel()
    
    global m; global s
    print m, s
    r = scipy.stats.linregress(m, s)    
    costs = abs(1-r.rvalue)
    
    print 'band: ',band,'\n','costs:', costs, 'rvalue: ', r.rvalue, 'slop: ', r.slope,'inter',r.intercept, '\n', 'parameters: ', p,'\n'
    if (r.intercept<0) or (r.slope>1):
        costs = costs*1000000000000000.
    return costs

def ransaclin(x,y):
    y, x = y.reshape((len(y),1)), x.reshape((len(x),1))
    model = linear_model.LinearRegression()
    model.fit(x, y)

    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(x, y)
    inlier_mask = model_ransac.inlier_mask_
    return x[inlier_mask], y[inlier_mask]

def ScaleExtent(data, shape): # used for unifine different array,

    re = int(shape[0]/(data.shape[0]))

    a = np.repeat(np.repeat(data, re, axis = 1), re, axis =0)
    
    if (re*(data.shape[0])-shape[0]) != 0:
        extended = np.zeros(shape)
        extended[:re*(data.shape[0]),:re*(data.shape[0])] = a
        extended[re*(data.shape[0]):,re*(data.shape[0]):] = a[re*(data.shape[0])-shape[0]:, re*(data.shape[0])-shape[0]]
        return extended
    else:
        return a

def get_psf(sent, sinds, mod, minds, band):
    p = np.array([23.9744718775, 306.15258185, 7.91598096945, -21.5616564408, 59.7537708998])
    psolve = optimize.fmin(cost,p,full_output=1, args=(sent,sinds, mod, minds, band))
    print 'solved b%02d: '%band, psolve
    return [band,psolve]

def op(ind,  args=None ):
    fpath, sentm, brdfs, sinds, minds = args
    Sent = gdal_read(bands[ind], fpath)[keys[ind]]
    sent = ScaleExtent(Sent, (10980,10980)) 
    sent[sentm]=100000000000
    sent[np.isnan(sent)] = 100000000000
    if ind<4:
        brdfs[ind][brdfs[ind].mask] = np.nan                
        psolve = get_psf(sent,sinds, brdfs[ind]*0.001, minds, bands[ind])

    else:
        brdfs[ind-1][brdfs[ind-1].mask] = np.nan
        psolve = get_psf(sent,sinds, brdfs[ind-1]*0.001, minds, bands[ind])
    return psolve

<ipython-input-33-b97ec1a94d58>:76: SyntaxWarning: name 'm' is assigned to before global declaration
  global m; global s
<ipython-input-33-b97ec1a94d58>:76: SyntaxWarning: name 's' is assigned to before global declaration
  global m; global s


In [36]:
fpath = spath

In [37]:
sentm = get_cloud_mask(fpath)
doy = int(mfile[0].split('.')[1][-3:])
pos = fpath.split('/')[6]+fpath.split('/')[7]+fpath.split('/')[8]
if sentm.sum()/(10980.*10980.) <0.2:
    print 'DOY: ', doy,'\n', 'Location: ', pos, 
    print 'Cloud proportion: ', sentm.sum()/(10980.*10980.)
    minds, sinds = get_coords(lat,lon) 

    modis_filenames = gdal.Open(mfile[0]).GetSubDatasets()
    modisQA = gdal.Open(mfile[1]).GetSubDatasets()

    brdfs = get_rs(modisQA, modis_filenames, fpath)
    args = fpath, sentm, brdfs, sinds, minds
    par = partial(op, args=args)
    pool = multiprocessing.Pool(processes = 7)
    retval = pool.map(par, range(7))
    pool.close()
    pool.join()
    #print doy, lat, lon, retval
    parallel_rw_pkl(retval, '%s%spsfs'%(pos, doy), 'w')
    if ret:
        parallel_rw_pkl([m,s], '%s%s%to_regs'%(pos, doy), 'w')

else:
    print 'Too much cloud, and this tile (doy: %s, lat: %s, lon: %s) is considered as invalid.'%(doy, lat, lon)
        


Here is the cloud probability calculation!

/home/ucfafyi/DATA/Sentinel/s_data/21/M/TT/2016/1/9/0/ (2, 3, 4, 8, 11, 12, 13)
No meteo data!!!
7%
14%
21%
29%
36%
43%
50%
57%
64%
71%
79%
86%
93%
99%
Done!!!
DOY:  9 
Location:  21MTT Cloud proportion:  0.194853890664


/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in greater
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in less
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in greater
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in less
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in greater
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in 

[0.0443004501577842 0.0433004501577842 0.04753643443541395 ...,
 0.04467534385063145 0.04616661400420784 0.04383514106058936][0.04205258735645178 0.04304070883274912 0.04105258735645178 ...,
 0.04158061880119228 0.04058061880119228 0.043227895959326836][0.1423547810468448 0.14533102399943948 0.14109503972180973 ...,
 0.14704419589796672 0.14704419589796672 0.1535313727059612] [ 0.09344861  0.07789653  0.0660125  ...,  0.04571667  0.04438889  0.042175  ]
 [ 0.12610764  0.12031111  0.08920929 ...,  0.08441111  0.07913611
  0.07724444]
 [ 0.17292292  0.17432361  0.17172465 ...,  0.12309444  0.1232      0.11952222]
band:  12 
band:  3 
band:  11 
costs: 0.543802826689 rvalue:  0.456197173311 slop:  6.41489433276 inter -0.225717709947 
costs: 0.502973572889 rvalue:  0.497026427111 slop:  5.81276692068 inter -0.151979096978 
costs: 1.17013307397 rvalue:  -0.170133073974 slop:  -1.4922245886 inter 0.364693500618 
parameters:  [  23.97447188  306.15258185    7.91598097  -21.56165644   59.75377

/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in greater
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in less


[0.2931039031888506 0.29509202466514795 0.2941039031888506 ...,
 0.29119177554509224 0.29702138286767304 0.30196578045455946] [ 0.30678056  0.31120278  0.30937057 ...,  0.24415     0.24606667
  0.23968889]


/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in greater
/data/store01/data_dirs/students/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in less


band:  8 
costs: 0.703354264932 rvalue:  0.296645735068 slop:  2.23158924356 inter -0.403762397618 
parameters:  [  23.97447188  306.15258185    7.91598097  -21.56165644   59.7537709 ] 
[0.023392148618373184 0.024392148618373185 0.02538027009467052 ...,
 0.02392018006311368 0.02292018006311368 0.021932058586816346] [ 0.1091125   0.09914861  0.09675208 ...,  0.05789306  0.05280278
  0.05107778]

band:  4 
costs: 0.583330758459 rvalue:  0.416669241541 slop:  7.47139374354 inter -0.115312500734 
parameters:  [  23.97447188  306.15258185    7.91598097  -21.56165644   59.7537709 ] 



MemoryError: 

In [ ]:
m